In [53]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Read the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv( "weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
0,0,Mahebourg,MU,1/5/2020 11:19,-20.41,57.70,81.00,83,75,8.05,0.00,0.0,broken clouds
1,1,Hobart,AU,1/5/2020 11:18,-42.88,147.33,60.80,55,20,11.41,0.00,0.0,few clouds
2,2,Yellowknife,CA,1/5/2020 11:15,62.45,-114.38,-2.00,84,90,13.87,0.00,0.0,light snow
3,3,Avarua,CK,1/5/2020 11:19,-21.21,-159.78,77.00,94,100,3.36,0.00,0.0,overcast clouds
4,4,Turukhansk,RU,1/5/2020 11:19,65.80,87.96,18.19,94,100,16.42,1.44,0.0,light snow


In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to preference raining/snowing or not? 
raining = str(input("Do you want it to be raining? (yes/no)"))
snowing = str(input("Do you want it to be snowing? (yes/no)"))


What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)yes


In [25]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[( ((city_data_df["Max Temp"] <= max_temp)) & 
                                       ( (city_data_df["Max Temp"] >= min_temp))    ) &
                                       ( 
                                           ((raining == 'yes') & (city_data_df["Rain inches(last 3hrs)"] > 0.0)) |
                                           ((snowing == 'yes') & (city_data_df["Snow inches(last 3hrs)"] > 0.0)) |
                                           
                                           ((raining == 'no') & (city_data_df["Rain inches(last 3hrs)"] == 0.0)) |
                                           ((snowing == 'no') & (city_data_df["Snow inches(last 3hrs)"] == 0.0))  
                                       )
                                      ]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
4,4,Turukhansk,RU,1/5/2020 11:19,65.80,87.96,18.19,94,100,16.42,1.44,0.00,light snow
7,7,Mataura,NZ,1/5/2020 11:19,-46.19,168.86,50.00,77,64,11.01,0.00,0.31,light rain
8,8,Kavieng,PG,1/5/2020 11:19,-2.57,150.80,84.88,73,65,11.10,0.00,3.50,moderate rain
22,22,Kaliua,TZ,1/5/2020 11:19,-5.06,31.79,84.24,57,90,1.68,0.00,3.12,moderate rain
28,28,Atuona,PF,1/5/2020 11:19,-9.80,-139.03,79.92,71,3,5.37,0.00,0.31,light rain


In [26]:
preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Country"] == "RU") ]
                                              # |
                                      #(city_data_df["City_ID"] == "38") |
                                      #(city_data_df["City_ID"] == "38") ]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
4,4,Turukhansk,RU,1/5/2020 11:19,65.80,87.96,18.19,94,100,16.42,1.44,0.0,light snow
83,83,Yerbogachen,RU,1/5/2020 11:19,61.28,108.01,13.62,92,100,14.00,0.56,0.0,light snow
86,86,Dikson,RU,1/5/2020 11:19,73.51,80.55,12.47,88,100,22.77,0.50,0.0,light snow
296,296,Severo-Kurilsk,RU,1/5/2020 11:20,50.68,156.12,19.02,92,100,9.46,0.94,0.0,light snow
547,547,Moshenskoye,RU,1/5/2020 11:21,58.52,34.59,29.30,93,93,8.75,0.31,0.0,light snow


In [54]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(60)
# hotel_df.loc[(hotel_df["Country"] == 'US')

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
4,Turukhansk,RU,18.19,65.80,87.96,light snow,
83,Yerbogachen,RU,13.62,61.28,108.01,light snow,
86,Dikson,RU,12.47,73.51,80.55,light snow,
296,Severo-Kurilsk,RU,19.02,50.68,156.12,light snow,
547,Moshenskoye,RU,29.30,58.52,34.59,light snow,


In [45]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping."+str(row["City"]))

Hotel not found... skipping.Yerbogachen
Hotel not found... skipping.Dikson
Hotel not found... skipping.Moshenskoye


In [46]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [49]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

directions1 = gmaps.directions_layer(locations.iloc[0], locations.iloc[1], stroke_color='red', \
                                    stroke_weight=3.0, \
                                    stroke_opacity=1.0,  travel_mode='DRIVING')

directions2 = gmaps.directions_layer(locations.iloc[1], locations.iloc[2], stroke_color='red', \
                                    stroke_weight=3.0, \
                                    stroke_opacity=1.0,  travel_mode='DRIVING')
directions3 = gmaps.directions_layer(locations.iloc[2], locations.iloc[3], stroke_color='red', \
                                    stroke_weight=3.0, \
                                    stroke_opacity=1.0,  travel_mode='DRIVING')

fig.add_layer(directions1)
fig.add_layer(directions2)
fig.add_layer(directions3)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# WeatherPy_vacation.csv 
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")  
#
